# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

1. We would like to know if the average calling rate is significantly different for blacking-sounding names and white-sounding names. We are going to use two-sample t test as we dealt with mean.CLT applies as the sample size is larger than 30. 
2. H0: means of calling from two groups are equal; Ha: means of calling are different

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

In [16]:
# Your solution to Q3 here

# Compute margin of error, confidence interval, and p-value. 
# using both the bootstrapping and the frequentist statistical approaches.

# bootstrapping approach

def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data,func)

    return bs_replicates

# Compute mean of all calls
mean_call = np.mean(data.call)

# observed mean diff from two samples
obs_mean_diff = np.mean(w.call) - np.mean(b.call)

# Generate shifted arrays
w_shifted = w.call - np.mean(w.call) + mean_call
b_shifted = b.call - np.mean(b.call) + mean_call

# Compute 10,000 bootstrap replicates from shifted arrays
bs_replicates_w = draw_bs_reps(w_shifted, np.mean, size=10000)
bs_replicates_b = draw_bs_reps(b_shifted, np.mean, size=10000)

# Get replicates of difference of means: bs_replicates
bs_replicates = bs_replicates_w - bs_replicates_b

# Compute std and mean
bs_replicates_std = np.std(bs_replicates)
bs_replicates_mean = np.mean(bs_replicates)

# Compute and print p-value: p
p = np.sum(bs_replicates >= obs_mean_diff) / len(bs_replicates)
print('bootstrap method : p-value =', p)
print("p-value < 0.05, we reject H0. "
      "Therefore, calling rate for both groups are not the same")
# calculate the margin of error and confidence interval

n_w = len(w.call)
n_b = len(b.call)
call_diff_std_sample = np.sqrt(np.var(w.call)/n_w + np.var(b.call)/n_b)

# df: degree of freedom
df = n_w + n_b -2
t = obs_mean_diff/call_diff_std_sample

print("t= " + str(round(t,4)))

p = 1 - stats.t.cdf(t,df=df)
print("p-value: " + str(round(2*p,4)))

# calculate the margin of error
margin_of_err = 2*(bs_replicates_std/np.sqrt(len(bs_replicates)))*t
print("margin_of_err using bootstrap:" +str(round(margin_of_err,4)))

# confidence interval
ci = [round(bs_replicates_mean-margin_of_err,4),round(bs_replicates_mean+margin_of_err,4)]
print("confidence interval using bootstrap: " + str(ci))

# Cross Checking with the internal scipy function
t_stat, pvalue = stats.ttest_ind(w.call,b.call)

print("t statistics using internal scipy function/frequentist method: " +str(round(t_stat,4)))
print("p value using internal scipy function/frequentist method: " + str(round(pvalue,4)))

bootstrap method : p-value = 0.0
p-value < 0.05, we reject H0. Therefore, calling rate for both groups are not the same
t= 4.1156
p-value: 0.0
margin_of_err using bootstrap:0.0006
confidence interval using bootstrap: [-0.0005, 0.0008]
t statistics using internal scipy function/frequentist method: 4.1147
p value using internal scipy function/frequentist method: 0.0


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

The statistical analysis suggests that the calling rates of white-sounding names and black-soudning names are significantly different. 

This result does not mean that race/name is the most important factor in callback success as other factors such as education, years of experience etc might also be responsible to the calling rates. To eliminate other factors besides race/name, we need to take two samples of white-sounding names and black-sounding names, and keep all the other factors(e.g. education, years of experience) the same. Then we do the hypothesis testing on these two samples to check if there is a significant difference in the calling rates between two groups. 

We also conduct the hypothesis testing on two groups only having a difference in education(e.g. bachelors and masters) to check if there is a significant difference in the calling rates between two groups holding different levels of education.

We then conduct the hypothesis testings on other factors separately. Each hypothesis testing gives a p-value. Comparing the p-values could provide some suggestions on which factor is the most important to the calling rate.